# Importing and Installing Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip install catboost

     |████████████████████████████████| 64.4MB 66kB/s 


In [0]:
from catboost import *

In [0]:
df1 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/train.csv")
df2 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/test.csv")
valid = pd.read_csv("/content/drive/My Drive/PDDA/Part_Of_Expected_Results/real_result_20perc.csv")

In [0]:
# remove all rows that contains missing value
df1.replace(['-999', -999], np.nan, inplace=True)
df1.dropna(axis=0, inplace=True)

df2.replace(['-999', -999], np.nan, inplace=True)
df2.dropna(axis=0, inplace=True)

# Making DTC prediction with CatBoostRegressor

In [0]:
df1.nunique()

CAL      7458
CNC      5454
GR      20274
HRD     14223
HRM     15439
PE      10406
ZDEN     5734
DTC     20182
DTS     20323
dtype: int64

In [0]:
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTC

In [0]:
test = valid.drop(['DTC','DTS'],axis=1)

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor( n_estimators=500,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True,)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test)
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 18.2176992	test: 18.2176992	test1: 33.2242478	best: 33.2242478 (0)	total: 53.5ms	remaining: 26.7s
100:	learn: 3.0585720	test: 3.0585720	test1: 5.1619225	best: 5.1619225 (100)	total: 604ms	remaining: 2.38s
200:	learn: 2.6546443	test: 2.6546443	test1: 5.4954759	best: 5.1181044 (133)	total: 1.15s	remaining: 1.71s
300:	learn: 2.4011454	test: 2.4011454	test1: 5.7966948	best: 5.1181044 (133)	total: 1.72s	remaining: 1.14s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 5.118104436
bestIteration = 133

Shrink model to first 134 iterations.
err:  5.118104449348483
0:	learn: 18.1427686	test: 18.1427686	test1: 33.9201538	best: 33.9201538 (0)	total: 6.18ms	remaining: 3.08s
100:	learn: 3.0072758	test: 3.0072758	test1: 3.5390574	best: 3.5270374 (88)	total: 555ms	remaining: 2.19s
200:	learn: 2.6080030	test: 2.6080030	test1: 3.4603170	best: 3.4603170 (200)	total: 1.09s	remaining: 1.63s
300:	learn: 2.3726923	test: 2.3726923	test1: 3.5489890	best: 3.4599329 (201)	total: 1.66

4.365247153145562

In [0]:
cat_DTC_preds = np.mean(y_pred_totcb2, 0)

In [0]:
cat_DTC_preds

array([69.01963332, 94.70456328, 92.90480481, ..., 92.27737981,
       62.09626799, 71.36284705])

# Checking Catboost Performance with Validation Blind set Data

In [0]:
real_DTC_values = valid['DTC']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTC_values, cat_DTC_preds))))

Root Mean Square Error is: 4.29563


# Making DTC Prediction on Submission Data

In [0]:
results = pd.DataFrame(columns=['DTC','DTS'])

In [0]:
y_pred_totcb1=[]

In [0]:
p1 = m2.predict(df2)
y_pred_totcb1.append(p1)

In [0]:
sub_cat_dtc_pred =  np.mean(y_pred_totcb1, 0)

In [0]:
results['DTC'] = sub_cat_dtc_pred

In [0]:
results.head()

,DTC,DTS
0,102.714747,NaN
1,101.761868,NaN
2,103.348231,NaN
3,103.414831,NaN
4,104.958243,NaN


In [0]:
results['DTC'].min(), results['DTC'].max()

(55.82331061632066, 117.09102305868753)

In [0]:
results.shape

(11088, 2)

In [0]:
results.nunique()

DTC    10893
DTS        0
dtype: int64

# Making DTS prediction with XG_BOOST REGRESSOR

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [0]:
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTS

In [0]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [0]:
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.8606,
                 gamma=4.794,                 
                 learning_rate=0.07,
                 max_depth=7,
                 min_child_weight=14.03,
                 n_estimators=1000,                                                                    
                 subsample=0.9863,
                 seed=1234)

In [0]:
best_xgb_model.fit(X,y)

[14:21:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8606, gamma=4.794,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=14.03, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1234, silent=None, subsample=0.9863, verbosity=1)

In [0]:
xgb_DTS_preds = best_xgb_model.predict(test)

# Checking Xgboost_Performance on Predicting DTS

In [0]:
real_DTS_values = valid['DTS']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, xgb_DTS_preds))))

Root Mean Square Error is: 24.32433


# Checking Overall performance of both models

In [0]:
results = pd.DataFrame(columns=['DTC','DTS'])

In [0]:
results['DTC'] = cat_DTC_preds
results['DTS'] = xgb_DTS_preds
results.head()

,DTC,DTS
0,69.019633,126.749466
1,94.704563,175.521484
2,92.904805,182.808304
3,77.862650,155.265884
4,79.660607,151.914185


In [0]:
results_data = np.array(results)

In [0]:
valid_data = np.array(valid)
y_real = valid_data[:,-2:]

In [0]:
 print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(y_real, results))))

Root Mean Square Error is: 17.46604


# Making DTS Prediction on Submission Data

In [0]:
y_pred_totcb3=[]

In [0]:
p3 = best_xgb_model.predict(df2)
y_pred_totcb3.append(p3)

In [0]:
sub_xgb_dts_pred =  np.mean(y_pred_totcb3, 0)

In [0]:
results['DTS'] = sub_xgb_dts_pred

In [0]:
results.head()

,DTC,DTS
0,102.714747,210.978226
1,101.761868,212.262375
2,103.348231,215.751892
3,103.414831,212.604477
4,104.958243,218.427734


In [0]:
"#teamname_submission_#num.csv"

In [0]:
results.to_csv('TEAM_TRIUMPHANT_submission_01.csv', index=False)